In [ ]:
import os
import sys

from dotenv import find_dotenv, load_dotenv

if os.path.dirname(find_dotenv()) not in sys.path:
    sys.path.append(os.path.dirname(find_dotenv()))

from src.utils import llm as llm_util
from src.utils import debug as debug_util

load_dotenv(override=True)
VERBOSE = debug_util.string_to_bool(os.environ.get("VERBOSE"))
llm_openai = llm_util.create_llm("openai", "gpt-3.5-turbo", verbose=VERBOSE)

## Agent

In [ ]:
import uuid

import boto3
from langchain_core.tools import tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage


bedrock_agent_runtime_client = boto3.client(
    service_name="bedrock-agent-runtime", region_name=os.environ.get("AWS_DEFAULT_REGION")
)
agent_id = "R93VRMZXMC"
session_id = uuid.uuid4().hex


@tool
def query_aws(question: str) -> str:
    """Useful to answer questions about EC2 instance.

    :param question: An EC2 relevant question.
    :return: The answer to the queston.
    """
    response = bedrock_agent_runtime_client.invoke_agent(
        agentId=agent_id,
        agentAliasId="TSTALIASID",
        sessionId=session_id,
        inputText=question,
    )

    result = ""

    for event in response.get("completion"):
        chunk = event["chunk"]
        result += chunk["bytes"].decode()

    return result


tools = [TavilySearchResults(max_results=1), query_aws]

react_agent = create_react_agent(llm_openai, tools)

In [ ]:
question = "How can I create an EC2 instance?"
# question = "Find information about AWS Community Day New Zealand 2024"

res = react_agent.invoke({"messages": [HumanMessage(content=question)]})

In [ ]:
res["messages"][-1].pretty_print()

In [ ]:
config = {"configurable": {"thread_id": "abc123"}}
for events in react_agent.stream({"messages": [HumanMessage(content=question)]}, config):
    for key, value in events.items():
        print(f"Output from node '{key}':")
        print(value["messages"][-1].pretty_print())
    print("\n---\n")